In [1]:
import torch
import torchaudio
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import fairseq 
from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

2024-06-20 12:47:26 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [3]:
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([str(Path.home())+'/Daniyal/profanity_detection/model/kannada_pretrained_1400h.pt'])
model = model[0]
model.eval()

/home/ubuntu/miniconda3/envs/MLAlgos/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Wav2Vec2Model(
  (feature_extractor): ConvFeatureExtractionModel(
    (conv_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (1): Dropout(p=0.0, inplace=False)
        (2): Sequential(
          (0): TransposeLast()
          (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (2): TransposeLast()
        )
        (3): GELU(approximate='none')
      )
      (1-4): 4 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (1): Dropout(p=0.0, inplace=False)
        (2): Sequential(
          (0): TransposeLast()
          (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (2): TransposeLast()
        )
        (3): GELU(approximate='none')
      )
      (5-6): 2 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (1): Dropout(p=0.0, inplace=False)
        (2): Sequential(
          (0): TransposeLast()
          (1): Fp32Laye

In [4]:
class AudioDataset (Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd. read_csv (csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self) :
        return len(self.annotations)

    def __getitem__(self, index) :

        # 16000 * 30
        file_path = os.path.join(self. root_dir, self.annotations. iloc[index, 0])
        waveform, sr = torchaudio.load(file_path)


        #waveform= truncate_or_pad_waveform(waveform, target_length=100000)
        
        if self .annotations. iloc [index, 1]=='Yes':
            y_label = torch. tensor(1)
        else:
            y_label = torch. tensor (0)
        
        if self.transform:
            waveform = self.transform(waveform)
        
        
        waveform=model.feature_extractor(waveform)
        #max_pool, _ = torch.max(waveform, dim=2)

        return (waveform,self.annotations. iloc[index, 0], y_label)

In [11]:
hindi_test_dataset = AudioDataset (csv_file = '/home/ubuntu/Daniyal/profanity_detection/data/Hindi_test.csv', root_dir = "/home/ubuntu/Daniyal/profanity_detection/data/Prima/SC_audio_Hindi")

In [12]:
hindi_train_loader = DataLoader (dataset=hindi_test_dataset, batch_size=1, shuffle=True)

In [13]:
len(hindi_train_loader)

369

In [14]:
#for hindi dataset

labels=[]
names=[]
for batch_idx, (data, name, label) in enumerate(hindi_train_loader):


    fromatted_data= data.cpu().detach().numpy().reshape(512, -1)
    
    # # For demonstration, let's stop after 5 batches
    # if batch_idx >= 4:
    #     break

    labels.append(label.cpu().detach().numpy()[0])
    names.append(name[0].split('.')[0]+'.npy')
    print(label, name)

    np.save('/home/ubuntu/Daniyal/profanity_detection/data/hindi_features/test/'+name[0].split('.')[0]+'.npy', fromatted_data)
    # if batch_idx >= 4:
    #     break

np_csv = {
    "Name": names,
    "Label": labels
}
np_df = pd.DataFrame(np_csv)

np_df.to_csv('/home/ubuntu/Daniyal/profanity_detection/data/hindi_features/train/hindi_test.csv', index=False)

tensor([1]) ('Abuse_1197.wav',)
tensor([0]) ('Abuse_589.wav',)
tensor([0]) ('Abuse_327.wav',)
tensor([0]) ('Abuse_800.wav',)
tensor([0]) ('Abuse_1179.wav',)
tensor([0]) ('Abuse_714.wav',)
tensor([1]) ('Abuse_295.wav',)
tensor([1]) ('Abuse_335.wav',)
tensor([1]) ('Abuse_490.wav',)
tensor([1]) ('Abuse_889.wav',)
tensor([1]) ('Abuse_475.wav',)
tensor([0]) ('Abuse_176.wav',)
tensor([0]) ('Abuse_1171.wav',)
tensor([1]) ('Abuse_942.wav',)
tensor([1]) ('Abuse_14.wav',)
tensor([0]) ('Abuse_439.wav',)
tensor([1]) ('Abuse_864.wav',)
tensor([0]) ('Abuse_477.wav',)
tensor([1]) ('Abuse_134.wav',)
tensor([1]) ('Abuse_783.wav',)
tensor([0]) ('Abuse_424.wav',)
tensor([1]) ('Abuse_1062.wav',)
tensor([0]) ('Abuse_731.wav',)
tensor([0]) ('Abuse_1098.wav',)
tensor([0]) ('Abuse_646.wav',)
tensor([0]) ('Abuse_286.wav',)
tensor([1]) ('Abuse_168.wav',)
tensor([1]) ('Abuse_272.wav',)
tensor([1]) ('Abuse_270.wav',)
tensor([0]) ('Abuse_756.wav',)
tensor([1]) ('Abuse_921.wav',)
tensor([1]) ('Abuse_118.wav',)
tens

In [17]:
temp=pd.read_csv('/home/ubuntu/Daniyal/profanity_detection/data/hindi_features/test/hindi_test.csv')
temp

,Name,Label
0,Abuse_1197.npy,1
1,Abuse_589.npy,0
2,Abuse_327.npy,0
3,Abuse_800.npy,0
4,Abuse_1179.npy,0
...,...,...
364,Abuse_248.npy,1
365,Abuse_104.npy,0
366,Abuse_433.npy,1
367,Abuse_834.npy,1
